In [ ]:
import os

from dotenv import load_dotenv
from mistralai import Mistral

In [ ]:
load_dotenv()

client = Mistral(api_key=os.getenv('MISTRAL_API_KEY'))

In [ ]:
uploaded_file = client.files.upload(
      file = {
         "file_name" : "tendermint.pdf",  # Or the name of the pdf you want to process 
         "content": open("tendermint.pdf", "rb")

      },
      purpose="ocr"
)

In [ ]:
file_url = client.files.get_signed_url (file_id = uploaded_file.id)

In [ ]:
response = client.ocr.process(
       model = "mistral-ocr-latest",
       document = {
           "type": "document_url",
           "document_url": file_url.url
       },
       include_image_base64 = True
)

In [ ]:
# response.pages[0].markdown

In [ ]:
import base64

def data_uri_to_bytes(data_uri):
    _, encoded = data_uri.split(',', 1)
    return base64.b64decode(encoded)

def export_image(image):
    parsed_image = data_uri_to_bytes(image.image_base64)
    with open(image.id, 'wb') as file:
        file.write(parsed_image)

with open('output.md', 'w') as f:
    for page in response.pages:
        f.write(page.markdown)
        for image in page.images:
            export_image(image)

In [ ]:
import pytesseract
from pdf2image import convert_from_bytes

with open('tendermint.pdf', 'rb' ) as f:
    pdf_bytes = f.read()

    images = convert_from_bytes (pdf_bytes)

    custom_config = r' --oem 3 --psm 6'

    pdf_text = '\n\n'.join(
        pytesseract.image_to_string(image, config=custom_config) for image in images
    )

    pdf_text = pdf_text.replace( '\x00', ' ')

In [ ]:
# print (pdf_text) 

In [ ]:
response = client.ocr.process (
    model = "mistral-ocr-latest",
    document = {
        "type": "document_url",
        "document_url": "https://blog.getzep.com/content/files/2025/01/ZEP__USING_KNOWLEDGE_GRAPHS_TO_POWER_LLM_AGENT_MEMORY_2025011700.pdf" # Replace link with link to your pdf
    },
    include_image_base64 = True
)

In [ ]:
# response.pages[0].markdown